In [121]:
import pandas as pd
import numpy as np
import ta
from datetime import datetime, timedelta

In [122]:
# INPUT DATA
filename1='/Users/lulu/Documents/STOCK/DATASET/DAILY/daily_data_2025-02-23.feather'
df=pd.read_feather(filename1)
# Ensure the data is sorted by date for each stockcode group
df = df.sort_values(by=['stockcode', 'date'])
df

,stockcode,stockname,date,open,close,high,low,volume,amount,outstanding_share,turnover
3072,sh600000,浦发银行,2021-01-04,9.64,9.69,9.73,9.55,62906939.0,606364959.0,2.935214e+10,0.002143
3073,sh600000,浦发银行,2021-01-05,9.68,9.68,9.68,9.52,53859177.0,517135737.0,2.935214e+10,0.001835
3074,sh600000,浦发银行,2021-01-06,9.62,9.82,9.83,9.62,61881321.0,604071110.0,2.935214e+10,0.002108
3075,sh600000,浦发银行,2021-01-07,9.83,9.81,9.93,9.66,57090442.0,557051920.0,2.935214e+10,0.001945
3076,sh600000,浦发银行,2021-01-08,9.83,9.83,9.90,9.73,55801455.0,547035205.0,2.935214e+10,0.001901
...,...,...,...,...,...,...,...,...,...,...,...
2979252,sz003816,中国广核,2025-02-17,3.69,3.80,3.80,3.66,228002600.0,851777561.0,3.933499e+10,0.005796
2979253,sz003816,中国广核,2025-02-18,3.79,3.76,3.84,3.75,157419350.0,596229027.0,3.933499e+10,0.004002
2979254,sz003816,中国广核,2025-02-19,3.75,3.74,3.77,3.71,123493969.0,461123903.0,3.933499e+10,0.003140
2979255,sz003816,中国广核,2025-02-20,3.73,3.72,3.75,3.70,94057840.0,350226633.0,3.933499e+10,0.002391


In [123]:
def analyze_stock_data(df, roc_period=5, rsi_window=6, drop_percentage=5):
    # 1. Calculate body size, upper wick, and lower wick
    df['body_size'] = abs(df['close'] - df['open'])
    df['upper_wick'] = df['high'] - df[['close', 'open']].max(axis=1)
    df['lower_wick'] = df[['close', 'open']].min(axis=1) - df['low']
    df['long_wick'] = (
    (df['upper_wick'] > 1 * df['body_size']) |  # Upper wick is long
    (df['lower_wick'] > 4 * df['body_size'])    # Lower wick is long
    )   

    # 2. Price downtrend (ROC for close price)
    col_pricetrend = f'price_down_{drop_percentage}'
    df[col_pricetrend] = ta.momentum.ROCIndicator(df['close'], window=roc_period).roc()

    # 3. RSI (Relative Strength Index)
    rsi_column_name = f'rsi_{rsi_window}'
    df[rsi_column_name] = ta.momentum.RSIIndicator(df['close'], window=rsi_window).rsi()

    # 4. Add 'vol_trend' for volume trend (Rate of Change for volume)
    col_voltrend = f'vol_down_{drop_percentage}'
    df[col_voltrend] = ta.momentum.ROCIndicator(df['volume'], window=roc_period).roc()

    # Shift the vol_trend by 1 day to get the previous day's volume trend relative to hammer day
    df['prev_day_vol_trend'] = df[col_voltrend].shift(1)

    # 5. Add 'vol_spikes' for volume spike (ROC for volume)
    df['volspikes'] = ta.momentum.ROCIndicator(df['volume'], window=1).roc()

    # 6. Bullish Engulfing Pattern (body comparison between current and previous day)
    df['body_size_previous'] = abs(df['close'].shift(1) - df['open'].shift(1))

    # Remove rows where 'stockcode' is NaN (if required)
    df = df[df['stockcode'].notna()]

    return df

In [124]:
#31s
df = analyze_stock_data(df)
df

,stockcode,stockname,date,open,close,high,low,volume,amount,outstanding_share,...,body_size,upper_wick,lower_wick,long_wick,price_down_5,rsi_6,vol_down_5,prev_day_vol_trend,volspikes,body_size_previous
3072,sh600000,浦发银行,2021-01-04,9.64,9.69,9.73,9.55,62906939.0,606364959.0,2.935214e+10,...,0.05,0.04,0.09,False,NaN,NaN,NaN,NaN,NaN,NaN
3073,sh600000,浦发银行,2021-01-05,9.68,9.68,9.68,9.52,53859177.0,517135737.0,2.935214e+10,...,0.00,0.00,0.16,True,NaN,NaN,NaN,NaN,-14.382773,0.05
3074,sh600000,浦发银行,2021-01-06,9.62,9.82,9.83,9.62,61881321.0,604071110.0,2.935214e+10,...,0.20,0.01,0.00,False,NaN,NaN,NaN,NaN,14.894665,0.00
3075,sh600000,浦发银行,2021-01-07,9.83,9.81,9.93,9.66,57090442.0,557051920.0,2.935214e+10,...,0.02,0.10,0.15,True,NaN,NaN,NaN,NaN,-7.742044,0.20
3076,sh600000,浦发银行,2021-01-08,9.83,9.83,9.90,9.73,55801455.0,547035205.0,2.935214e+10,...,0.00,0.07,0.10,True,NaN,NaN,NaN,NaN,-2.257798,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2979252,sz003816,中国广核,2025-02-17,3.69,3.80,3.80,3.66,228002600.0,851777561.0,3.933499e+10,...,0.11,0.00,0.03,False,3.260870,78.349900,46.948109,2.732213,69.093202,0.01
2979253,sz003816,中国广核,2025-02-18,3.79,3.76,3.84,3.75,157419350.0,596229027.0,3.933499e+10,...,0.03,0.05,0.01,True,2.452316,64.190579,46.896451,46.948109,-30.957213,0.11
2979254,sz003816,中国广核,2025-02-19,3.75,3.74,3.77,3.71,123493969.0,461123903.0,3.933499e+10,...,0.01,0.02,0.03,True,1.630435,57.911186,34.135618,46.896451,-21.550960,0.03
2979255,sz003816,中国广核,2025-02-20,3.73,3.72,3.75,3.70,94057840.0,350226633.0,3.933499e+10,...,0.01,0.02,0.02,True,0.813008,51.827235,-27.963269,34.135618,-23.836086,0.01


In [125]:
def condition_stocks(df):
    # Initialize the 'group' column with None
    df['group'] = None

    # Initialize individual condition dataframes
    df_bullish = df.copy()
    df_hammer = df.copy()
    df_insidebar = df.copy()
    df_momentum = df.copy()
    df_multiple_longwick = df.copy()

    # Bullish Engulfing Condition
    conditions_bullish = (
        (df_bullish['price_down_5'] <= -5) & 
        (df_bullish['rsi_6'] <= 39) & 
        (df_bullish['volspikes'] >= 30) &
        (df_bullish['close'].shift(1) < df_bullish['open'].shift(1)) & 
        (df_bullish['close'] > df_bullish['open']) & 
        (df_bullish['open'] < df_bullish['close'].shift(1)) & 
        (df_bullish['close'] > df_bullish['open'].shift(1)) & 
        (df_bullish['body_size'] > 1.3 * df_bullish['body_size'].shift(1))  # Bullish engulfing condition
    )
    df_bullish.loc[conditions_bullish, 'group'] = 'bullishgolf'

    # Hammer Candlestick Condition
    conditions_hammer = (
        (df_hammer['price_down_5'] <= -5) & 
        (df_hammer['rsi_6'] <= 39) & 
        (df_hammer['prev_day_vol_trend'] <= -4) & 
        (df_hammer['volspikes'] >= 50) &
        (df_hammer['lower_wick'] >= 2 * df_hammer['body_size']) & 
        (df_hammer['upper_wick'] <= 0.9 * df_hammer['body_size']) & 
        (df_hammer['body_size'] > 0)  # Hammer candlestick pattern
    )
    df_hammer.loc[conditions_hammer, 'group'] = 'hammer'

    # Inside Bar Condition
    conditions_insidebar = (
        (df_insidebar['price_down_5'] <= -5) & 
        (df_insidebar['rsi_6'] <= 38) &
        (df_insidebar['high'] < df_insidebar['high'].shift(1)) & 
        (df_insidebar['low'] > df_insidebar['low'].shift(1)) &
        (df_insidebar['close'].shift(1) < df_insidebar['open'].shift(1)) &  # Day 1 is bearish
        (df_insidebar['close'] > df_insidebar['open']) &  # Day 2 is bullish
        (df_insidebar['close'] < df_insidebar['open'].shift(1)) &  # Day 2 close < Day 1 open
        (df_insidebar['close'].shift(1) < df_insidebar['open'])  # Day 1 close < Day 2 open
    )
    df_insidebar.loc[conditions_insidebar, 'group'] = 'insidebar'

    # Momentum Candle Condition
    conditions_momentum = (
        (df_momentum['price_down_5'] <= -4) & 
        (df_momentum['rsi_6'] <= 39) & 
        (df_momentum['close'] > df_momentum['open']) & 
        (df_momentum['close'].shift(1) < df_momentum['open'].shift(1)) & 
        (df_momentum['body_size'] > 1.5 * df_momentum['body_size'].shift(1)) & 
        ((df_momentum['close'] - df_momentum['close'].shift(1)) / df_momentum['close'].shift(1) * 100 >= 2.2)  # Momentum candle condition
    )
    df_momentum.loc[conditions_momentum, 'group'] = 'momentum'

    # Multiple Long Wick Condition
    conditions_multiple_long_wick = (
        (df_multiple_longwick['price_down_5'] <= -5) & 
        (df_multiple_longwick['rsi_6'] <= 39) &
        ((df_multiple_longwick['upper_wick'] > 1 * df_multiple_longwick['body_size']) | 
         (df_multiple_longwick['lower_wick'] > 4 * df_multiple_longwick['body_size'])) &  # Long wick condition
        (df_multiple_longwick['long_wick'] & df_multiple_longwick['long_wick'].shift(1) & df_multiple_longwick['long_wick'].shift(2)) &  # Consecutive long wick candles
        ((df_multiple_longwick['close'] / df_multiple_longwick['close'].shift(1) - 1).abs() <= 0.03) &
        ((df_multiple_longwick['close'] / df_multiple_longwick['close'].shift(2) - 1).abs() <= 0.03)  # Price range condition
    )
    df_multiple_longwick.loc[conditions_multiple_long_wick, 'group'] = 'multiplelongwick'

    # Filter only the rows that satisfy each condition and add a new 'condition' column to track
    result_bullish = df_bullish[['date', 'stockcode', 'stockname', 'group']].dropna(subset=['group'])

    result_hammer = df_hammer[['date', 'stockcode', 'stockname', 'group']].dropna(subset=['group'])

    result_insidebar = df_insidebar[['date', 'stockcode', 'stockname', 'group']].dropna(subset=['group'])

    result_momentum = df_momentum[['date', 'stockcode', 'stockname', 'group']].dropna(subset=['group'])

    result_multiple_longwick = df_multiple_longwick[['date', 'stockcode', 'stockname', 'group']].dropna(subset=['group'])

    # Combine all the datasets into one
    combined_result = pd.concat([result_bullish, result_hammer, result_insidebar, result_momentum, result_multiple_longwick], axis=0)

    # Reset the index of the combined result DataFrame
    combined_result = combined_result.reset_index(drop=True)

    return combined_result


In [126]:
result=condition_stocks(df)
result

,date,stockcode,stockname,group
0,2022-07-27,sh600011,华能国际,bullishgolf
1,2022-04-27,sh600017,日照港,bullishgolf
2,2021-12-01,sh600029,南方航空,bullishgolf
3,2022-03-11,sh600031,三一重工,bullishgolf
4,2022-09-21,sh600031,三一重工,bullishgolf
...,...,...,...,...
17791,2024-04-12,sz003042,中农联合,multiplelongwick
17792,2021-05-25,sz003043,华亚智能,multiplelongwick
17793,2022-03-10,sz003043,华亚智能,multiplelongwick
17794,2022-09-20,sz003043,华亚智能,multiplelongwick


In [ ]:
# Step 3: Get today's date (end_date for fetching new data)
today_date='2025-02-23'
#today_date = datetime.today().strftime('%Y-%m-%d')
today_date



'2025-02-22'

In [140]:
output=result[(result['date']==today_date)]
output

,date,stockcode,stockname,group


In [120]:
filename = f'/Users/lulu/Documents/STOCK/DAILY_STOCK/STOCK_{today_date}.csv'
output.to_csv(filename, index=False, encoding='utf-8-sig')